In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from gmdhpy import gmdh
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import random


df_original = pd.read_csv("./wind_data/wind_dataset.csv")
df_original["DATE"] = pd.to_datetime(df_original["DATE"], errors='coerce')
df_original.interpolate(method='linear', inplace=True)
df_original["TEMP_DIFF"] = df_original["T.MAX"] - df_original["T.MIN"]

features = ["RAIN", "T.MAX", "T.MIN", "T.MIN.G", "TEMP_DIFF"]
target = "WIND"

X = df_original[features].values
y = np.log1p(df_original[target].values)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Train GMDH model
model = gmdh.MultilayerGMDH()
model.param.ref_function = gmdh.RefFunctionType.rfQuadratic
model.param.criterion_type = gmdh.CriterionType.cmpTest
model.param.max_layer_count = 100
model.param.min_layer_size = 50
model.param.max_layer_size = 500
model.fit(X_train, y_train)

print("✅ Wind prediction model trained and ready!")

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n📊 Model Evaluation:")
print(f"MAE : {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R²  : {r2*100:.2f}%")

joblib.dump(model, "gmdh_wind_model.pkl")


def predict_wind_user_input():
    while True:
        print("\n🌬️ Enter meteorological values to predict WIND speed.")
        try:
            rain = float(input("RAIN (mm): "))
            t_max = float(input("T.MAX (°C): "))
            t_min = float(input("T.MIN (°C): "))
            t_min_g = float(input("T.MIN.G (°C): "))
        except ValueError:
            print("⚠️ Invalid input. Please enter numeric values.")
            continue

        temp_diff = t_max - t_min
        full_input = [rain, t_max, t_min, t_min_g, temp_diff]

        match = df_original[
            (df_original["RAIN"] == rain) &
            (df_original["T.MAX"] == t_max) &
            (df_original["T.MIN"] == t_min) &
            (df_original["T.MIN.G"] == t_min_g) &
            (df_original["TEMP_DIFF"] == temp_diff)
        ]

        print("\n📥 Input received:")
        print(f"RAIN: {rain}, T.MAX: {t_max}, T.MIN: {t_min}, T.MIN.G: {t_min_g}")

       
            input_array = np.array(full_input).reshape(1, -1)
            input_scaled = scaler.transform(input_array)
            predicted_log = model.predict(input_scaled)[0]
            predicted_wind = np.expm1(predicted_log)
            print(f"YPredicted WIND speed: {predicted_wind:.2f} m/s")

    
        cont = input("\n🔁 Predict again? (y/n): ").strip().lower()
        if cont != "y":
            print("👋 Exiting prediction loop. Goodbye!")
            break

# Start the prediction loop
predict_wind_user_input()


train layer0 in 0.18 sec
train layer1 in 0.96 sec
train layer2 in 0.97 sec
train layer3 in 0.99 sec
train layer4 in 1.06 sec
train layer5 in 1.02 sec
✅ Wind prediction model trained and ready!

📊 Model Evaluation:
MAE : 0.383
RMSE: 0.486
R²  : 11.09%

🌬️ Enter meteorological values to predict WIND speed.


RAIN (mm):  3.5
T.MAX (°C):  11.3
T.MIN (°C):  7.3
T.MIN.G (°C):  3.8



📥 Input received:
RAIN: 3.5, T.MAX: 11.3, T.MIN: 7.3, T.MIN.G: 3.8
Predicted WIND speed: 18.50 m/s



🔁 Predict again? (y/n):  y



🌬️ Enter meteorological values to predict WIND speed.


RAIN (mm):  5
T.MAX (°C):  9
T.MIN (°C):  3
T.MIN.G (°C):  3.2



📥 Input received:
RAIN: 5.0, T.MAX: 9.0, T.MIN: 3.0, T.MIN.G: 3.2
YPredicted WIND speed: 11.75 m/s



🔁 Predict again? (y/n):  n


👋 Exiting prediction loop. Goodbye!
